In [1]:
!pip install tensorflow

In [ ]:
import pandas as pd
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
medium_data = pd.read_csv('/content/drive/MyDrive/medium_data.csv')
medium_data.head()

In [ ]:
print('Number of records: ', medium_data.shape[0])
print('Number of fields: ', medium_data.shape[1])

In [ ]:
medium_data['title']

In [ ]:
#the lambda function takes each title (referred to as X) and replaces occurrences of the unicode character \xaO with standard space
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'\xa0', u' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace('\u200a', ' '))

In [ ]:
tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(medium_data['title'])
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)
print("Word: ID")
print("------------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print("Strong: ", tokenizer.word_index['strong'])
print("And: ", tokenizer.word_index['and'])
print("Consumption: ", tokenizer.word_index['consumption'])

In [ ]:
input_sequences = []
for line in medium_data['title']:
  token_list = tokenizer.texts_to_sequences([line])[0]
  #print(token_list)

  for i in range(1,len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

#print(input_sequences)
print('Total input sequences: ', len(input_sequences))

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[1]

In [ ]:
#create features and label
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(xs[5])
print(labels[5])
print(ys[5][14])

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))#150 specifies the number of units (or neurons) in the LSTM layer that is being used in a bidirectional
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=50, verbose=1)
#print model.summary()
print(model)

In [ ]:
#predicting the next word of the title
seed_text = 'implementation of'
next_words = 2

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted_probs = model.predict(token_list, verbose=0)
  predicted = np.argmax(predicted_probs, axis=-1)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word
print(seed_text)